<a href="https://colab.research.google.com/github/aymankhchman/RAG-llama3/blob/main/Building_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we'll build a RAG from scratch. In the first part we'll use jaccard simliarity then use embedding showing how embedding works better at capturing the context. Finally, we'll ask llama3 to act as lebanese chef a provide us a menu giving what we're craving. i.e: a salad, something refreshing, dish with high protein ect ...

## Our data will be about lebanese food

In [ ]:
dishes = [
    {
        "name": "Hummus",
        "ingredients": ["chickpeas", "tahini", "lemon juice", "garlic"],
        "description": "A creamy dip made from blended chickpeas, tahini, lemon juice, and garlic, usually served with pita bread.",
    },
    {
        "name": "Tabbouleh",
        "ingredients": ["parsley", "bulgur", "tomatoes", "mint", "onions", "lemon juice", "olive oil"],
        "description": "A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.",
    },
    {
        "name": "Falafel",
        "ingredients": ["chickpeas", "onions", "garlic", "parsley", "spices"],
        "description": "Deep-fried balls made from ground chickpeas, onions, garlic, parsley, and spices, typically served in pita bread.",
    },
    {
        "name": "Baba Ghanoush",
        "ingredients": ["eggplants", "tahini", "lemon juice", "garlic", "olive oil"],
        "description": "A smoky dip made from roasted eggplants, tahini, lemon juice, garlic, and olive oil, often served with pita bread.",
    },
    {
        "name": "Shawarma",
        "ingredients": ["chicken", "beef/lamb", "yogurt", "spices", "garlic", "lemon juice"],
        "description": "Marinated and spit-roasted meat, usually served in pita bread with vegetables and sauces.",
    },
    {
        "name": "Kibbeh",
        "ingredients": ["bulgur", "ground meat", "onions", "pine nuts", "spices"],
        "description": "A dish made of bulgur and ground meat, often shaped into balls or patties and stuffed with spiced meat and pine nuts.",
    },
    {
        "name": "Mujadara",
        "ingredients": ["lentils", "rice", "onions", "spices"],
        "description": "A hearty dish made with lentils, rice, and caramelized onions, seasoned with spices.",
    },
    {
        "name": "Fattoush",
        "ingredients": ["lettuce", "tomatoes", "cucumbers", "radishes", "onions", "fried pita bread", "sumac", "lemon juice"],
        "description": "A fresh salad made with mixed greens, vegetables, and fried pita bread, seasoned with sumac and lemon juice.",
    },
    {
        "name": "Kafta",
        "ingredients": ["ground meat", "parsley", "onions", "spices"],
        "description": "Grilled skewers made from ground meat mixed with parsley, onions, and spices, typically served with rice or bread.",
    },
    {
        "name": "Stuffed Grape Leaves (Warak Enab)",
        "ingredients": ["grape leaves", "rice", "ground meat", "tomatoes", "spices"],
        "description": "Grape leaves stuffed with a mixture of rice, ground meat, tomatoes, and spices, then cooked in a pot.",
    },
    {
        "name": "Baklava",
        "ingredients": ["phyllo dough", "nuts", "honey", "sugar", "butter"],
        "description": "A sweet pastry made with layers of phyllo dough, filled with nuts and sweetened with honey and sugar syrup.",
    },
    {
        "name": "Labneh",
        "ingredients": ["yogurt", "salt"],
        "description": "A creamy strained yogurt cheese, often served with olive oil, za'atar, and pita bread.",
    },
    {
        "name": "Manakish",
        "ingredients": ["dough", "za'atar", "cheese", "ground meat"],
        "description": "A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.",
    },
    {
        "name": "Fatayer",
        "ingredients": ["dough", "spinach", "cheese", "meat"],
        "description": "Savory pastries filled with spinach, cheese, or meat, often served as appetizers or snacks.",
    },
    {
        "name": "Sambousek",
        "ingredients": ["dough", "ground meat", "onions", "pine nuts", "spices"],
        "description": "Fried or baked pastries filled with spiced ground meat, onions, and pine nuts.",
    },
    {
        "name": "Kibbeh Nayeh",
        "ingredients": ["raw ground meat", "bulgur", "onions", "spices"],
        "description": "A raw meat dish made from ground meat, bulgur, onions, and spices, served with pita bread.",
    },
    {
        "name": "Shish Tawook",
        "ingredients": ["chicken", "yogurt", "lemon juice", "garlic", "spices"],
        "description": "Marinated chicken skewers grilled to perfection, often served with garlic sauce and pita bread.",
    },
    {
        "name": "Makdous",
        "ingredients": ["baby eggplants", "walnuts", "red peppers", "olive oil", "garlic"],
        "description": "Baby eggplants stuffed with walnuts, red peppers, and garlic, then preserved in olive oil.",
    },
    {
        "name": "Zaatar Bread",
        "ingredients": ["dough", "za'atar", "olive oil"],
        "description": "Flatbread topped with a mixture of za'atar and olive oil, baked until crispy.",
    },
    {
        "name": "Arayes",
        "ingredients": ["pita bread", "ground meat", "onions", "tomatoes", "spices"],
        "description": "Stuffed pita bread with spiced ground meat, onions, and tomatoes, grilled until crispy.",
    },
    {
        "name": "Knafeh",
        "ingredients": ["semolina", "cheese", "sugar syrup", "butter"],
        "description": "A sweet dessert made with layers of semolina dough and cheese, soaked in sugar syrup.",
    },
    {
        "name": "Moussaka",
        "ingredients": ["eggplants", "tomatoes", "onions", "chickpeas", "spices"],
        "description": "A vegetarian dish made with layers of eggplants, tomatoes, onions, and chickpeas, seasoned with spices.",
    },
    {
        "name": "Foul Mudammas",
        "ingredients": ["fava beans", "lemon juice", "garlic", "olive oil", "cumin"],
        "description": "A warm fava bean dish seasoned with lemon juice, garlic, olive oil, and cumin, often served for breakfast.",
    },
    {
        "name": "Bulgur Pilaf",
        "ingredients": ["bulgur", "tomatoes", "onions", "chickpeas", "spices"],
        "description": "A hearty pilaf made with bulgur, tomatoes, onions, chickpeas, and spices, often served as a side dish.",
    },
    {
        "name": "Maamoul",
        "ingredients": ["semolina", "dates", "walnuts", "pistachios", "sugar"],
        "description": "Shortbread cookies filled with dates, walnuts, or pistachios, typically enjoyed during holidays.",
    },
    {
        "name": "Shakshuka",
        "ingredients": ["eggs", "tomatoes", "peppers", "onions", "spices"],
        "description": "A dish of poached eggs in a spicy tomato and pepper sauce, often served for breakfast or brunch.",
    },
    {
        "name": "Samkeh Harra",
        "ingredients": ["fish", "tahini", "garlic", "lemon juice", "chili peppers"],
        "description": "Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.",
    },
    {
        "name": "Fasolia",
        "ingredients": ["white beans", "tomatoes", "onions", "garlic", "spices"],
        "description": "A bean stew made with white beans, tomatoes, onions, and garlic, seasoned with spices.",
    },
    {
        "name": "Roz Bil Halib",
        "ingredients": ["rice", "milk", "sugar", "rose water", "pistachios"],
        "description": "A creamy rice pudding flavored with rose water and topped with crushed pistachios."
    }
  ]


### functions to process the text data

In [ ]:
def text_to_words(text: str) -> set[str]:
  return set(text.lower().split())


def jaccard_similarity(query, text) -> float:
  query_words = text_to_words(query)
  text_words = text_to_words(text)

  intersection = query_words.intersection(text_words)
  union = query_words.union(text_words)

  return len(intersection)/len(union)

In [ ]:
def retrieve(query, documents, k: int=3):
  scores = [(jaccard_similarity(query, doc), doc) for doc in documents]
  scores.sort(reverse=True, key=lambda x: x[0])
  return scores[:k]

## Test our simple code here

In [ ]:
corpus = [
    f'{e["name"]} (ingredients: {", ".join(e["ingredients"])} ). {e["description"]}'
    for e in dishes
]

In [ ]:
corpus[:2]

['Hummus (ingredients: chickpeas, tahini, lemon juice, garlic ). A creamy dip made from blended chickpeas, tahini, lemon juice, and garlic, usually served with pita bread.',
 'Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.']

In [ ]:
retrieve("fresh salad with a citruis taste", corpus, k=3)

[(0.17391304347826086,
  'Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.'),
 (0.14285714285714285,
  'Fattoush (ingredients: lettuce, tomatoes, cucumbers, radishes, onions, fried pita bread, sumac, lemon juice ). A fresh salad made with mixed greens, vegetables, and fried pita bread, seasoned with sumac and lemon juice.'),
 (0.1,
  'Mujadara (ingredients: lentils, rice, onions, spices ). A hearty dish made with lentils, rice, and caramelized onions, seasoned with spices.')]

## Integrating LLM now

In [ ]:
!pip --quiet install transformers
!pip --quiet install accelerate

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline
)


In [ ]:
from huggingface_hub import login
login()

Next we'll use LLAMA3-8B-Instruct as our Model

In [ ]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype=torch.bfloat16)

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.repitition_penalty = 1.24
generation_config.temperature = 0.001
generation_config.do_sample = True

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer
)

In [ ]:
prompt = """
Write a lebanese dish that contain beef or lamb.
{dish_text}

Make sure the receipe is respect the traditional receipe.
"""

In [ ]:
found_dishes = retrieve(prompt, corpus, k=10)

In [ ]:
found_dishes

[(0.08571428571428572,
  "Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza."),
 (0.075,
  'Shakshuka (ingredients: eggs, tomatoes, peppers, onions, spices ). A dish of poached eggs in a spicy tomato and pepper sauce, often served for breakfast or brunch.'),
 (0.07142857142857142,
  'Kibbeh (ingredients: bulgur, ground meat, onions, pine nuts, spices ). A dish made of bulgur and ground meat, often shaped into balls or patties and stuffed with spiced meat and pine nuts.'),
 (0.0625,
  'Mujadara (ingredients: lentils, rice, onions, spices ). A hearty dish made with lentils, rice, and caramelized onions, seasoned with spices.'),
 (0.058823529411764705,
  'Moussaka (ingredients: eggplants, tomatoes, onions, chickpeas, spices ). A vegetarian dish made with layers of eggplants, tomatoes, onions, and chickpeas, seasoned with spices.'),
 (0.05405405405405406,
  'Kibbeh Nayeh (ingredients: r

In [ ]:
dishes_text = "\n\n".join([e for _, e in found_dishes])
prompt_text = prompt.format(dish_text=dishes_text)

In [ ]:
print(prompt_text)

"\nWrite a lebanese dish that contain beef or lamb.\nManakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.\n\nShakshuka (ingredients: eggs, tomatoes, peppers, onions, spices ). A dish of poached eggs in a spicy tomato and pepper sauce, often served for breakfast or brunch.\n\nKibbeh (ingredients: bulgur, ground meat, onions, pine nuts, spices ). A dish made of bulgur and ground meat, often shaped into balls or patties and stuffed with spiced meat and pine nuts.\n\nMujadara (ingredients: lentils, rice, onions, spices ). A hearty dish made with lentils, rice, and caramelized onions, seasoned with spices.\n\nMoussaka (ingredients: eggplants, tomatoes, onions, chickpeas, spices ). A vegetarian dish made with layers of eggplants, tomatoes, onions, and chickpeas, seasoned with spices.\n\nKibbeh Nayeh (ingredients: raw ground meat, bulgur, onions, spices ). A raw meat dish made from ground me

In [ ]:
SYSTEM_PROMPT = """
You're an expert Lebanese Chef with over 10+ years of experience working in the Kitchen. You provide the method of cooking lebanese dishes with details while giving tips and tricks to achieve a great dish
"""

prompt_text = SYSTEM_PROMPT + prompt_text

In [ ]:
messages = [
    {"role":"user", "content": prompt_text}
]

prompt_test = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt_test)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You're an expert Lebanese Chef with over 10+ years of experience working in the Kitchen. You provide the method of cooking lebanese dishes with details while giving tips and tricks to achieve a great dish

Write a lebanese dish that contain beef or lamb.
Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.

Shakshuka (ingredients: eggs, tomatoes, peppers, onions, spices ). A dish of poached eggs in a spicy tomato and pepper sauce, often served for breakfast or brunch.

Kibbeh (ingredients: bulgur, ground meat, onions, pine nuts, spices ). A dish made of bulgur and ground meat, often shaped into balls or patties and stuffed with spiced meat and pine nuts.

Mujadara (ingredients: lentils, rice, onions, spices ). A hearty dish made with lentils, rice, and caramelized onions, seasoned with spices.

Moussaka (ingredients: eggplants

In [ ]:
output = llm(prompt_test)

I'm delighted to share with you a traditional Lebanese recipe that features lamb as the main ingredient. Here's a recipe for a classic dish that's sure to please:

**Kibbeh Bil Sanieh (Lamb Kibbeh with Pine Nuts and Spices)**

Ingredients:

* 1 pound ground lamb
* 1/2 cup bulgur
* 1/4 cup pine nuts
* 1/4 cup chopped onion
* 2 cloves garlic, minced
* 1 teaspoon ground cumin
* 1 teaspoon ground coriander
* 1/2 teaspoon ground cinnamon
* Salt and black pepper, to taste
* 2 tablespoons olive oil
* 2 tablespoons lemon juice
* 2 tablespoons chopped fresh parsley

Instructions:

1. In a large mixing bowl, combine the ground lamb, bulgur, pine nuts, onion, garlic, cumin, coriander, cinnamon, salt, and black pepper. Mix well with your hands until the ingredients are well combined.
2. Gradually add the olive oil, lemon juice, and parsley to the mixture, mixing until the kibbeh forms a cohesive dough.
3. Cover the dough with plastic wrap and refrigerate for at least 30 minutes to allow the flavor

## Use Embeddings - Since the jaccard similarity doesn't take into account the context and perform poorly

In [ ]:
!pip install --quiet sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer


EMBEDDING_MODEL = "Alibaba-NLP/gte-large-en-v1.5"

embedding_model = SentenceTransformer(EMBEDDING_MODEL, trust_remote_code=True)

In [ ]:
prompt_template = """
Write a lebanese menu that is refreshing and have a citrus taste

{dish_text}

Make sure the receipe is respect the traditional receipe.
"""

In [ ]:
prompts = [prompt_template]
prompt_embeddings = embedding_model.encode(prompts, normalize_embeddings=True)

In [ ]:
prompt_embeddings.shape

(1, 1024)

In [ ]:
corpus_embeddings = embedding_model.encode(corpus, normalize_embeddings=True)

In [ ]:
similarities = corpus_embeddings @ prompt_embeddings.T

In [ ]:
print(similarities.ravel())

[0.54462206 0.6277888  0.5190695  0.57411796 0.5737035  0.48590523
 0.48676166 0.58356583 0.48734164 0.52201396 0.5448406  0.5945851
 0.5964895  0.47326076 0.463158   0.51820135 0.5597     0.53379023
 0.5130693  0.563637   0.47184724 0.514063   0.5371524  0.5010721
 0.46680072 0.51147103 0.65563715 0.45734632 0.5593186 ]


In [ ]:
scores = [(score[0], dish) for score, dish in zip(similarities, corpus)]
scores = sorted(scores, reverse=True, key=lambda x: x[0])
scores[:3]

[(0.65563715,
  'Samkeh Harra (ingredients: fish, tahini, garlic, lemon juice, chili peppers ). Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.'),
 (0.6277888,
  'Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.'),
 (0.5964895,
  "Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.")]

In [ ]:
dishes = "\n\n".join(e for _, e in scores[:3])

In [ ]:
print(dishes)

Samkeh Harra (ingredients: fish, tahini, garlic, lemon juice, chili peppers ). Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.

Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.

Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.


In [ ]:
prompt_text = prompt_template.format(dish_text=dishes)
print(prompt_text)


Write a lebanese menu that is refreshing and have a citrus taste

Samkeh Harra (ingredients: fish, tahini, garlic, lemon juice, chili peppers ). Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.

Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.

Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.

Make sure the receipe is respect the traditional receipe.



In [ ]:
prompt_text = SYSTEM_PROMPT + prompt_text

In [ ]:
print(prompt_text)


You're an expert Lebanese Chef with over 10+ years of experience working in the Kitchen. You provide the method of cooking lebanese dishes with details while giving tips and tricks to achieve a great dish

Write a lebanese menu that is refreshing and have a citrus taste

Samkeh Harra (ingredients: fish, tahini, garlic, lemon juice, chili peppers ). Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.

Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.

Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.

Make sure the receipe is respect the traditional receipe.



In [ ]:
messages = [
    {"role": "user",
     "content": prompt_text}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You're an expert Lebanese Chef with over 10+ years of experience working in the Kitchen. You provide the method of cooking lebanese dishes with details while giving tips and tricks to achieve a great dish

Write a lebanese menu that is refreshing and have a citrus taste

Samkeh Harra (ingredients: fish, tahini, garlic, lemon juice, chili peppers ). Spicy baked fish topped with a tahini, garlic, and lemon juice sauce, garnished with chili peppers.

Tabbouleh (ingredients: parsley, bulgur, tomatoes, mint, onions, lemon juice, olive oil ). A fresh salad made with finely chopped parsley, bulgur, tomatoes, mint, onions, lemon juice, and olive oil.

Manakish (ingredients: dough, za'atar, cheese, ground meat ). A type of Lebanese flatbread topped with za'atar, cheese, or ground meat, similar to a pizza.

Make sure the receipe is respect the traditional receipe.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




## Let's like now at what the menu the llama3 provides

In [ ]:
output_menu = llm(prompt)

The flavors of Lebanon! I'm delighted to share with you a refreshing and citrusy menu that showcases the country's culinary delights. Here's a traditional Lebanese menu that will transport your taste buds to the Mediterranean coast:

**Samkeh Harra (Spicy Baked Fish)**

This classic Lebanese dish is a staple in many households, and for good reason. The combination of tender fish, creamy tahini sauce, and a hint of spice is simply divine.

Ingredients:

* 4 fish fillets (any white fish works well, such as tilapia or cod)
* 1/2 cup tahini
* 3 cloves garlic, minced
* 2 tablespoons freshly squeezed lemon juice
* 1-2 chili peppers, seeded and finely chopped (depending on desired level of heat)
* Salt and pepper to taste
* Fresh parsley, chopped (optional)

Instructions:

1. Preheat your oven to 400°F (200°C).
2. In a blender or food processor, combine tahini, garlic, lemon juice, and chili peppers. Blend until smooth.
3. Place the fish fillets in a baking dish and brush the tahini sauce eve